In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams.update({"font.size": 12})
%matplotlib inline

In C:\Users\96ank\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\96ank\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\96ank\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\96ank\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases lat

In [ ]:
!pip3 install flair

In [2]:
df = pd.read_csv("data/WSB_top_users_features.csv", sep=";")
df.tail(1)

,Unnamed: 0,author,id,title,selftext,date,day,month,year,tickers_,...,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity,years_count,months_count,dates_count
7252,7252,zxc123zxc123,12,Is WSB hiding the scale of financial crisis? +...,There’s a bubble they’d like us to think WE ca...,"['2021-10-13', '2021-06-04', '2021-06-04', '20...",13 4 4 4 29 22 19 19 7 29 23 23,"['10', '6', '6', '6', '4', '1', '1', '1', '1',...","['2021', '2021', '2021', '2021', '2021', '2021...",NaN,...,0.515,0.59,-0.227381,0.561012,"['Is', 'WSB', 'hiding', 'the', 'scale', 'of', ...","['There', '’s', 'a', 'bubble', 'they', '’d', '...",0.287619,"{'2021': 9, '2020': 3}","{'10': 1, '6': 3, '4': 1, '1': 4, '12': 3}","{'2021-10-13': 1, '2021-06-04': 3, '2021-04-29..."


# Embedding of titles by user

In [ ]:
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings
from flair.data import Sentence
from sklearn.manifold import TSNE
%matplotlib inline


## Visualize Embeddings for clustering

## Stacked Embeddings

In [ ]:
from flair.embeddings import  FlairEmbeddings
# initialize the word embeddings
glove_embedding = WordEmbeddings('glove')

# initialize the document embeddings, mode = mean
document_embeddings = DocumentPoolEmbeddings([glove_embedding])
# init Flair forward and backwards embeddings
flair_embedding_forward = FlairEmbeddings('news-forward')
flair_embedding_backward = FlairEmbeddings('news-backward')

2021-12-26 17:36:36,015 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmplmlea279


100%|██████████| 160000128/160000128 [00:06<00:00, 25998870.03B/s]

2021-12-26 17:36:42,480 copying /tmp/tmplmlea279 to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2021-12-26 17:36:42,804 removing temp file /tmp/tmplmlea279
2021-12-26 17:36:43,323 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmpegklw76a


100%|██████████| 21494764/21494764 [00:01<00:00, 14792396.61B/s]

2021-12-26 17:36:45,096 copying /tmp/tmpegklw76a to cache at /root/.flair/embeddings/glove.gensim


2021-12-26 17:36:45,132 removing temp file /tmp/tmpegklw76a


In [ ]:
from flair.embeddings import StackedEmbeddings

# create a StackedEmbedding object that combines glove and forward/backward flair embeddings
stacked_embeddings = StackedEmbeddings([
                                        glove_embedding,
                                        flair_embedding_forward,
                                        flair_embedding_backward,
                                       ])

In [ ]:
def get_stackedsentence_emb(text):
  sentence = Sentence(text)
  stacked_embeddings.embed(sentence)
  return sentence.embedding


# now check out the embedded tokens.
#for token in sentence:
#    print(token)
#    print(token.embedding)

In [ ]:
#df['title_emb_stacked'] = df['title'].apply(lambda x: get_stackedsentence_emb(x))
title_emb_stacked = df.loc[:50,'title'].apply(lambda x: get_stackedsentence_emb(x))

In [ ]:
from sklearn.manifold import TSNE
%matplotlib inline
def plot_embeddings(df_name,label_name,emb_col_name,total_points):

	all_users = []
	all_users_emb = []
	for index, row in df_name.iterrows():
		 #print(row["author"], row["title_emb"])
		 all_users.append(row[label_name])
		 all_users_emb.append(row[emb_col_name].numpy())
		 

	tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
	new_values = tsne_model.fit_transform(all_users_emb)

	x = []
	y = []
	for value in new_values:
		x.append(value[0])
		y.append(value[1])
			
	plt.figure(figsize=(16, 16)) 
	for i in range(len(x[0:total_points])):
		plt.scatter(x[i],y[i])
		plt.annotate(all_users[i],
						 xy=(x[i], y[i]),
						 xytext=(5, 2),
						 textcoords='offset points',
						 ha='right',
						 va='bottom')
	plt.show()

In [ ]:
#plot_embeddings(df,"author","title_emb_stacked",100)

In [ ]:
#plot_embeddings(df[df['score'] > 250],"author","title_emb_stacked",50)

# Count Years, Months 

In [ ]:
from collections import Counter

In [ ]:
df['year'] = df['year'].apply(lambda x: list(x.split(" ")))
df["years_count"] = df['year'].apply(lambda x: dict(Counter(list(x))))

In [ ]:
df['month'] = df['month'].apply(lambda x: list(x.split(" ")))
df["months_count"] = df['month'].apply(lambda x: dict(Counter(list(x))))

In [ ]:
df['date'] = df['date'].apply(lambda x: list(x.split(" ")))
df["dates_count"] = df['date'].apply(lambda x: dict(Counter(list(x))))

In [ ]:
df.head(1)

,author,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity,years_count,months_count,dates_count
0,-A-Brocoma2021,14,"Strength of a silverback gorilla, dominance of...",What if everyone did? Bought just one more s...,"[2021-04-03, 2021-03-28, 2021-03-16, 2021-03-1...",3 28 16 16 10 10 10 27 26 25 25 23 20 2,"[4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2]","[2021, 2021, 2021, 2021, 2021, 2021, 2021, 202...",['GME'],{'GME': 1},Meme Discussion Meme YOLO Technical Analysis G...,5.428571,76,19.285714,270,0.972857,13.62,"['Meme', 'Discussion', 'Meme', 'YOLO', 'Techni...","{'Meme': 2, 'Discussion': 2, 'YOLO': 8, 'Techn...",0.22166,0.507253,0.25,0.25,"['Strength', 'of', 'a', 'silverback', 'gorilla...","[' ', 'What', 'if', 'everyone', 'did', '?', '...",0.47166,{'2021': 14},"{'4': 1, '3': 6, '2': 7}","{'2021-04-03': 1, '2021-03-28': 1, '2021-03-16..."


In [ ]:
df.to_csv("/content/drive/MyDrive/WSB/WSB_top_users_features.csv",sep=';')

## Are there differences between "old" users (pre-2021) and new ones?

In [ ]:
# split dataset: pre and post hype 
#users_pre = df['year'].apply(lambda x:list(set(x).intersection(pre_years)))
users_post = df[['2021' in x for x in df['year'].values] and ['2020' not in x for x in df['year'].values] and ['2018' not in x for x in df['year'].values] and ['2019' not in x for x in df['year'].values]]
users_post.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5919 entries, 0 to 7252
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              5919 non-null   object 
 1   id                  5919 non-null   int64  
 2   title               5919 non-null   object 
 3   selftext            5919 non-null   object 
 4   date                5919 non-null   object 
 5   day                 5919 non-null   object 
 6   month               5919 non-null   object 
 7   year                5919 non-null   object 
 8   tickers_            2497 non-null   object 
 9   tickers_count_      5919 non-null   object 
 10  link_flair_text     5919 non-null   object 
 11  comments_mean       5919 non-null   float64
 12  comments_sum        5919 non-null   int64  
 13  score_mean          5919 non-null   float64
 14  score_sum           5919 non-null   int64  
 15  upvote_ratio_mean   5919 non-null   float64
 16  upvote

In [ ]:
users_post.to_csv("/content/drive/MyDrive/WSB/WSB_users_post-hype.csv",sep=';')

In [ ]:
users_pre = df[['2021' not in x for x in df['year'].values]]
users_pre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1034 entries, 2 to 7250
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              1034 non-null   object 
 1   id                  1034 non-null   int64  
 2   title               1034 non-null   object 
 3   selftext            1034 non-null   object 
 4   date                1034 non-null   object 
 5   day                 1034 non-null   object 
 6   month               1034 non-null   object 
 7   year                1034 non-null   object 
 8   tickers_            448 non-null    object 
 9   tickers_count_      1034 non-null   object 
 10  link_flair_text     1034 non-null   object 
 11  comments_mean       1034 non-null   float64
 12  comments_sum        1034 non-null   int64  
 13  score_mean          1034 non-null   float64
 14  score_sum           1034 non-null   int64  
 15  upvote_ratio_mean   1034 non-null   float64
 16  upvote

In [ ]:
users_pre.head(1)

,author,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity,years_count,months_count,dates_count
2,-F1sh-,11,Very Bullish jobs chart. SPY 400 on the horizo...,https://twitter.com/espn_esports/s...,"[2020-12-04, 2020-09-08, 2020-09-07, 2020-09-0...",4 8 7 7 27 15 28 24 8 7 31,"[12, 9, 9, 9, 6, 6, 4, 4, 4, 4, 3]","[2020, 2020, 2020, 2020, 2020, 2020, 2020, 202...",NaN,{},Discussion Shitpost Meme Meme YOLO Meme Discus...,6.090909,67,1.0,11,0.545455,6.0,"['Discussion', 'Shitpost', 'Meme', 'Meme', 'YO...","{'Discussion': 2, 'Shitpost': 5, 'Meme': 3, 'Y...",0.022222,0.111111,0.074583,0.450407,"['Very', 'Bullish', 'jobs', 'chart', '.', 'SPY...","[' ', 'https://twitter.com/espn_esp...",0.096805,{'2020': 11},"{'12': 1, '9': 3, '6': 2, '4': 4, '3': 1}","{'2020-12-04': 1, '2020-09-08': 1, '2020-09-07..."


In [ ]:
users_pre.to_csv("/content/drive/MyDrive/WSB/WSB_users_pre-hype.csv",sep=';')

In [ ]:
users_pre_post = df[['2021' in x for x in df['year'].values] and ['2020' in x for x in df['year'].values] and ['2018' in x for x in df['year'].values] and ['2019' in x for x in df['year'].values]]
users_pre_post.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1334 entries, 4 to 7250
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              1334 non-null   object 
 1   id                  1334 non-null   int64  
 2   title               1334 non-null   object 
 3   selftext            1334 non-null   object 
 4   date                1334 non-null   object 
 5   day                 1334 non-null   object 
 6   month               1334 non-null   object 
 7   year                1334 non-null   object 
 8   tickers_            693 non-null    object 
 9   tickers_count_      1334 non-null   object 
 10  link_flair_text     1334 non-null   object 
 11  comments_mean       1334 non-null   float64
 12  comments_sum        1334 non-null   int64  
 13  score_mean          1334 non-null   float64
 14  score_sum           1334 non-null   int64  
 15  upvote_ratio_mean   1334 non-null   float64
 16  upvote

In [ ]:
users_pre_post.head(1)

,author,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity,years_count,months_count,dates_count
4,-Hyre,19,The age of silver. And clandestine inflation. ...,You're all front row to the Fed money printing...,"[2021-01-29, 2021-01-29, 2021-01-28, 2019-12-0...",29 29 28 4 2 14 24 18 17 9 8 7 3 3 3 2 27 23 20,"[1, 1, 1, 12, 12, 11, 10, 10, 10, 10, 10, 10, ...","[2021, 2021, 2021, 2019, 2019, 2019, 2019, 201...","['AMZN', 'ROKU', 'MU', 'BYND', 'GME']","{'AMZN': 1, 'ROKU': 1, 'MU': 1, 'BYND': 1, 'GM...",DD Discussion DD YOLO Options YOLO Discussion ...,333.842105,6343,164.315789,3122,0.156316,2.97,"['DD', 'Discussion', 'DD', 'YOLO', 'Options', ...","{'DD': 3, 'Discussion': 2, 'YOLO': 5, 'Options...",-0.15,0.3,0.036943,0.479711,"['The', 'age', 'of', 'silver', '.', 'And', 'cl...","['You', ""'re"", 'all', 'front', 'row', 'to', 't...",-0.113057,"{'2021': 3, '2019': 16}","{'1': 3, '12': 2, '11': 1, '10': 10, '9': 3}","{'2021-01-29': 2, '2021-01-28': 1, '2019-12-04..."


In [ ]:
users_pre_post.to_csv("/content/drive/MyDrive/WSB/WSB_users_pre-post_hype.csv",sep=';')

## Can we identify those that have posted (successful) DDs?

In [9]:
df.describe()

,Unnamed: 0,id,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,title_polarity,title_subjectivity,body_polarity,body_subjectivity,polarity
count,7253.000000,7253.000000,7253.000000,7.253000e+03,7253.000000,7253.000000,7253.000000,7253.000000,7253.000000,7253.000000,7253.000000,7253.000000,7253.000000
mean,3626.000000,20.604991,44.710142,4.327122e+03,56.210572,1349.207500,0.808659,16.371195,0.064442,0.481713,0.044302,0.371057,0.108743
std,2093.905084,26.956220,422.546416,2.017091e+05,348.667883,9493.127294,0.314847,23.826521,0.162926,0.129514,0.122352,0.220327,0.211182
min,0.000000,11.000000,0.000000,0.000000e+00,1.000000,11.000000,0.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.360069
25%,1813.000000,12.000000,5.454545,8.200000e+01,1.000000,16.000000,0.750000,10.750000,-0.020567,0.408123,0.000000,0.288889,-0.002833
50%,3626.000000,15.000000,15.276596,2.540000e+02,1.846154,35.000000,0.981818,13.000000,0.063131,0.482381,0.031632,0.450747,0.111765
75%,5439.000000,21.000000,33.111111,6.380000e+02,9.055556,177.000000,1.000000,18.000000,0.153328,0.556878,0.095408,0.507998,0.223496
max,7252.000000,1103.000000,20629.751004,1.374015e+07,9204.916667,368913.000000,1.000000,1103.000000,1.000000,1.000000,1.000000,1.000000,1.371719


In [10]:
# overall top users > median
users_above_median = df[df['score_mean'] >= 1.846154]
users_above_median.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3625 entries, 0 to 7252
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          3625 non-null   int64  
 1   author              3625 non-null   object 
 2   id                  3625 non-null   int64  
 3   title               3625 non-null   object 
 4   selftext            3625 non-null   object 
 5   date                3625 non-null   object 
 6   day                 3625 non-null   object 
 7   month               3625 non-null   object 
 8   year                3625 non-null   object 
 9   tickers_            1694 non-null   object 
 10  tickers_count_      3625 non-null   object 
 11  link_flair_text     3625 non-null   object 
 12  comments_mean       3625 non-null   float64
 13  comments_sum        3625 non-null   int64  
 14  score_mean          3625 non-null   float64
 15  score_sum           3625 non-null   int64  
 16  upvote

In [20]:
users_above_median.to_csv('data/WSB_users_Overall_top_median.csv',sep=';',index=False)

In [17]:
# overall top between 25 to 75 percentile
users_25to75 = df.loc[(df['score_mean'] >= 1) & (df['score_mean'] <= 9.055556)]
users_25to75.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5440 entries, 1 to 7252
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          5440 non-null   int64  
 1   author              5440 non-null   object 
 2   id                  5440 non-null   int64  
 3   title               5440 non-null   object 
 4   selftext            5440 non-null   object 
 5   date                5440 non-null   object 
 6   day                 5440 non-null   object 
 7   month               5440 non-null   object 
 8   year                5440 non-null   object 
 9   tickers_            2319 non-null   object 
 10  tickers_count_      5440 non-null   object 
 11  link_flair_text     5440 non-null   object 
 12  comments_mean       5440 non-null   float64
 13  comments_sum        5440 non-null   int64  
 14  score_mean          5440 non-null   float64
 15  score_sum           5440 non-null   int64  
 16  upvote

In [18]:
users_25to75.to_csv('data/WSB_users_25to75.csv',sep=';',index=False)

In [3]:
## get all users with 'DD' flair and high upvotes, positive sentiment 
users_DD = df[['DD' in x for x in df['flairs'].values]]
users_DD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3486 entries, 3 to 7251
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          3486 non-null   int64  
 1   author              3486 non-null   object 
 2   id                  3486 non-null   int64  
 3   title               3486 non-null   object 
 4   selftext            3486 non-null   object 
 5   date                3486 non-null   object 
 6   day                 3486 non-null   object 
 7   month               3486 non-null   object 
 8   year                3486 non-null   object 
 9   tickers_            1813 non-null   object 
 10  tickers_count_      3486 non-null   object 
 11  link_flair_text     3486 non-null   object 
 12  comments_mean       3486 non-null   float64
 13  comments_sum        3486 non-null   int64  
 14  score_mean          3486 non-null   float64
 15  score_sum           3486 non-null   int64  
 16  upvote

In [4]:
users_DD.describe()

,Unnamed: 0,id,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,title_polarity,title_subjectivity,body_polarity,body_subjectivity,polarity
count,3486.000000,3486.000000,3486.000000,3486.000000,3486.000000,3486.000000,3486.000000,3486.000000,3486.000000,3486.000000,3486.000000,3486.000000,3486.000000
mean,3611.849971,22.414515,40.946608,956.551061,52.485032,1302.780264,0.782659,17.232263,0.068253,0.476584,0.056647,0.421233,0.124901
std,2094.849369,19.637300,303.983454,5231.594807,313.397822,7793.067789,0.323352,16.522570,0.146659,0.123992,0.103772,0.171173,0.190003
min,3.000000,11.000000,0.000000,0.000000,1.000000,11.000000,0.000000,0.000000,-0.719658,0.000000,-0.600000,0.000000,-0.688058
25%,1786.000000,13.000000,7.867857,124.000000,1.000000,18.000000,0.637700,10.480000,-0.013092,0.405556,0.000000,0.411067,0.015983
50%,3624.500000,17.000000,18.677083,338.000000,2.083333,43.000000,0.974674,13.580000,0.064998,0.476131,0.056917,0.461113,0.122863
75%,5417.750000,24.000000,37.127404,776.500000,9.571115,204.750000,1.000000,20.000000,0.150314,0.548867,0.102322,0.506615,0.229794
max,7251.000000,310.000000,15854.500000,214709.000000,6190.416667,268232.000000,1.000000,223.000000,1.000000,1.000000,1.000000,1.000000,1.165128


In [7]:
# by median 
users_DD_gt_Median = users_DD[users_DD['score_mean'] >= 2.083333]
users_DD_gt_Median.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1746 entries, 4 to 7243
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          1746 non-null   int64  
 1   author              1746 non-null   object 
 2   id                  1746 non-null   int64  
 3   title               1746 non-null   object 
 4   selftext            1746 non-null   object 
 5   date                1746 non-null   object 
 6   day                 1746 non-null   object 
 7   month               1746 non-null   object 
 8   year                1746 non-null   object 
 9   tickers_            944 non-null    object 
 10  tickers_count_      1746 non-null   object 
 11  link_flair_text     1746 non-null   object 
 12  comments_mean       1746 non-null   float64
 13  comments_sum        1746 non-null   int64  
 14  score_mean          1746 non-null   float64
 15  score_sum           1746 non-null   int64  
 16  upvote

In [19]:
users_DD_gt_Median.to_csv("data/WSB_users_DD_median.csv",sep=';',index=False)

In [ ]:
users_DD_top = users_DD[users_DD['score_mean'] > 60]
users_DD_top.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293 entries, 4 to 7224
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              293 non-null    object 
 1   id                  293 non-null    int64  
 2   title               293 non-null    object 
 3   selftext            293 non-null    object 
 4   date                293 non-null    object 
 5   day                 293 non-null    object 
 6   month               293 non-null    object 
 7   year                293 non-null    object 
 8   tickers_            154 non-null    object 
 9   tickers_count_      293 non-null    object 
 10  link_flair_text     293 non-null    object 
 11  comments_mean       293 non-null    float64
 12  comments_sum        293 non-null    int64  
 13  score_mean          293 non-null    float64
 14  score_sum           293 non-null    int64  
 15  upvote_ratio_mean   293 non-null    float64
 16  upvote_

In [ ]:
users_DD.to_csv("/content/drive/MyDrive/WSB/WSB_users_DD.csv",sep=';')

## Can we rank the users according to their expertise?

In [ ]:
## rank users based on upvote ratio, num_comments etc.
## order by score_mean 
users_DD = users_DD.sort_values(by=['score_mean'],ascending=False)
users_DD.head()

,author,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity,years_count,months_count,dates_count
4346,armchairtycoon,12,"""Give my regards to your puppet master!"" 🐙 Ult...",,"[2021-02-23, 2020-12-29, 2020-12-16, 2020-08-0...",23 29 16 5 16 19 13 13 12 7 3 1,"[2, 12, 12, 8, 6, 5, 3, 3, 3, 3, 3, 2]","[2021, 2020, 2020, 2020, 2020, 2020, 2020, 202...",NaN,{},News Loss Meme DD Loss Loss Shitpost Meme Sati...,622.916667,7475,6190.416667,74285,0.484167,5.81,"['News', 'Loss', 'Meme', 'DD', 'Loss', 'Loss',...","{'News': 1, 'Loss': 4, 'Meme': 2, 'DD': 1, 'Sh...",-0.102405,0.568889,0.000000,0.000000,"['""', 'Give', 'my', 'regards', 'to', 'your', '...",[' '],-0.102405,"{'2021': 1, '2020': 11}","{'2': 2, '12': 2, '8': 1, '6': 1, '5': 1, '3': 5}","{'2021-02-23': 1, '2020-12-29': 1, '2020-12-16..."
6717,superanth,16,5 Biggest Hedge Fund Casualties of Reddit Wall...,"On the fundamentals side, they just did a bi...","[2021-07-06, 2021-05-19, 2021-05-19, 2021-04-2...",6 19 19 25 31 11 11 4 3 3 1 1 16 15 13 13,"[7, 5, 5, 4, 3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3]","[2021, 2021, 2021, 2021, 2021, 2021, 2021, 202...",NaN,{},News Technical Analysis Technical Analysis New...,328.250000,5252,6159.312500,98549,0.721875,11.55,"['News', 'Technical', 'Analysis', 'Technical',...","{'News': 7, 'Technical': 2, 'Analysis': 2, 'Di...",0.012363,0.461722,0.108222,0.423550,"['5', 'Biggest', 'Hedge', 'Fund', 'Casualties'...","[' ', 'On', 'the', 'fundamentals', 'side', ',...",0.120585,"{'2021': 12, '2020': 4}","{'7': 1, '5': 2, '4': 1, '3': 5, '2': 7}","{'2021-07-06': 1, '2021-05-19': 2, '2021-04-25..."
2160,Makitt,12,Not Even a Shooting Star Can Fix the Portfolio...,*Before I start BlackBerry...,"[2021-08-13, 2021-07-14, 2021-05-20, 2021-05-2...",13 14 20 20 20 6 3 3 13 13 2 1,"[8, 7, 5, 5, 5, 5, 5, 5, 3, 2, 2, 2]","[2021, 2021, 2021, 2021, 2021, 2021, 2021, 202...",NaN,{},Meme Meme Meme Meme Meme Meme Meme Meme Meme N...,504.083333,6049,5768.166667,69218,0.982500,11.79,"['Meme', 'Meme', 'Meme', 'Meme', 'Meme', 'Meme...","{'Meme': 9, 'News': 2, 'DD': 1}",0.059524,0.595635,-0.023125,0.384583,"['Not', 'Even', 'a', 'Shooting', 'Star', 'Can'...","[' ', '*', 'Before', 'I', '...",0.036399,{'2021': 12},"{'8': 1, '7': 1, '5': 6, '3': 1, '2': 3}","{'2021-08-13': 1, '2021-07-14': 1, '2021-05-20..."
1825,Jkelchner4,14,Coincidence? The resemblance between Balkan’s ...,My brother and I are...,"[2021-02-02, 2021-02-01, 2021-01-28, 2021-01-2...",2 1 28 27 25 25 18 17 17 15 15 13 13 10,"[2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[2021, 2021, 2021, 2021, 2021, 2021, 2021, 202...","['CAT', 'VALE']","{'CAT': 1, 'VALE': 1}",DD Gain News Gain Meme Meme Meme Meme Meme Dis...,247.571429,3466,4725.285714,66154,0.995000,13.93,"['DD', 'Gain', 'News', 'Gain', 'Meme', 'Meme',...","{'DD': 2, 'Gain': 2, 'News': 1, 'Meme': 6, 'Di...",0.104299,0.506150,-0.093939,0.575758,"['Coincidence', '?', 'The', 'resemblance', 'be...","[' ', 'My', 'brother'...",0.010360,{'2021': 14},"{'2': 2, '1': 12}","{'2021-02-02': 1, '2021-02-01': 1, '2021-01-28..."
3427,Stylux,12,Haven't memed in a bit. Let's go. GME MEGATHRE...,Continuation from [https://www.reddit.com/r/...,"[2021-07-20, 2021-03-10, 2021-02-11, 2021-02-0...",20 10 11 2 29 28 28 16 23 21 21 7,"[7, 3, 2, 2, 1, 1, 1, 11, 10, 3, 3, 11]","[2021, 2021, 2021, 2021, 2021, 2021, 2021, 202...","['XOM', 'LL']","{'XOM': 1, 'LL': 1}",YOLO Daily Discussion Daily Discussion Daily...,15854.500000,190254,4650.666667,55808,0.735000,8.82,"['YOLO', 'Daily', 'Discussion', '', '', 'Daily...","{'YOLO': 1, 'Daily': 3, 'Discussion': 3, '': 6...",-0.180556,0.378704,0.153771,0.420946,"['Have', ""n't"", 'memed', 'in', 'a', 'bit', '.'...","[' ', 'Continuation', 'from', '[', 'https://w...",-0.026784,"{'2021': 7, '2020': 4, '2019': 1}","{'7': 1, '3': 3

## Are the same users active in the other reviewed subreddits?

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
## compare users with investing and stocks

wsb = pd.read_csv("/content/drive/MyDrive/WSB/WSB_top_users_features.csv", sep=";")
wsb.tail(1)


,Unnamed: 0,author,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity,years_count,months_count,dates_count
7252,7252,zxc123zxc123,12,Is WSB hiding the scale of financial crisis? +...,There’s a bubble they’d like us to think WE ca...,"['2021-10-13', '2021-06-04', '2021-06-04', '20...",13 4 4 4 29 22 19 19 7 29 23 23,"['10', '6', '6', '6', '4', '1', '1', '1', '1',...","['2021', '2021', '2021', '2021', '2021', '2021...",NaN,{},Meme Gain Meme Gain News Meme Meme Meme Discus...,9.0,108,2.083333,25,0.980833,11.77,"['Meme', 'Gain', 'Meme', 'Gain', 'News', 'Meme...","{'Meme': 6, 'Gain': 2, 'News': 1, 'Discussion'...",0.515,0.59,-0.227381,0.561012,"['Is', 'WSB', 'hiding', 'the', 'scale', 'of', ...","['There', '’s', 'a', 'bubble', 'they', '’d', '...",0.287619,"{'2021': 9, '2020': 3}","{'10': 1, '6': 3, '4': 1, '1': 4, '12': 3}","{'2021-10-13': 1, '2021-06-04': 3, '2021-04-29..."


In [4]:
stocks = pd.read_csv("/content/drive/MyDrive/stocks/stocks_top_users_features.csv", sep=";")
stocks.tail(1)

,Unnamed: 0,author,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity,years_count,months_count,dates_count
984,984,zeroskater45,16,Beyond Meat Cash Flow Issue? What is your favo...,"Hi,\n\nI noticed beyond meat has been a fairly...","['2020-12-28', '2020-12-04', '2020-12-01', '20...",28 4 1 3 10 9 8 1 28 20 20 29 18 27 26 25,"['12', '12', '12', '9', '7', '5', '5', '5', '4...","['2020', '2020', '2020', '2020', '2020', '2020...",NaN,{},Ticker Discussion Discussion Question Discussi...,7.8125,125,1.0,16,0.5,8.0,"['Ticker', 'Discussion', 'Discussion', 'Questi...","{'Ticker': 2, 'Discussion': 5, 'Question': 1, ...",0.336443,0.457257,0.234165,0.509774,"['Beyond', 'Meat', 'Cash', 'Flow', 'Issue', '?...","['Hi', ',', '\n\n', 'I', 'noticed', 'beyond', ...",0.570608,{'2020': 16},"{'12': 3, '9': 1, '7': 1, '5': 3, '4': 3, '3':...","{'2020-12-28': 1, '2020-12-04': 1, '2020-12-01..."


In [8]:
investing = pd.read_csv("/content/drive/MyDrive/investing/investing_top_users_features.csv", sep=";")

In [30]:
investing.head()

,Unnamed: 0,author,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity,years_count,months_count,dates_count
0,0,007moonboundnxs,11,HBK-GoChain Announces Investment In Arabian Bo...,[removed] [removed] [removed] [removed] [remov...,"['2019-12-10', '2019-12-08', '2019-11-25', '20...",10 8 25 15 14 9 1 14 10 7 7,"['12', '12', '11', '10', '10', '10', '6', '3',...","['2019', '2019', '2019', '2019', '2019', '2019...",NaN,{},,0.090909,1,1.000000,11,0.000000,0.00,"['', '', '', '', '', '', '', '', '', '', '']",{'': 11},0.056404,0.213405,0.000000,0.000000,"['HBK', '-', 'GoChain', 'Announces', 'Investme...","['[', 'removed', ']', '[', 'removed', ']', '['...",0.056404,"{'2019': 8, '2018': 3}","{'12': 2, '11': 4, '10': 3, '6': 1, '3': 1}","{'2019-12-10': 1, '2019-12-08': 1, '2019-11-25..."
1,1,01Cloud01,18,Brokerage firms that offer auto stock buying f...,Hello all I have seemingly been on a endless m...,"['2021-06-12', '2021-06-11', '2020-10-04', '20...",12 11 4 26 2 1 31 30 25 19 16 3 2 1 1 24 24 7,"['6', '6', '10', '7', '4', '4', '3', '3', '3',...","['2021', '2021', '2020', '2020', '2020', '2020...",NaN,{},,35.222222,634,28.444444,512,0.222222,4.00,"['', '', '', '', '', '', '', '', '', '', '', '...",{'': 18},0.077500,0.275000,0.116614,0.436446,"['Brokerage', 'firms', 'that', 'offer', 'auto'...","['Hello', 'all', 'I', 'have', 'seemingly', 'be...",0.194114,"{'2021': 2, '2020': 12, '2019': 2, '2018': 2}","{'6': 3, '10': 1, '7': 1, '4': 3, '3': 9, '9': 1}","{'2021-06-12': 1, '2021-06-11': 1, '2020-10-04..."
2,2,036Gooddaysir036,18,AT&amp;T (NYSE: T) - A stock Guide/Review AT&a...,[removed] [removed] [removed] [removed] # Intr...,"['2020-12-06', '2020-12-06', '2020-12-06', '20...",6 6 6 6 28 28 28 28 28 21 21 21 20 3 1 24 24 24,"['12', '12', '12', '12', '11', '11', '11', '11...","['2020', '2020', '2020', '2020', '2020', '2020...",NaN,{},,18.833333,339,1.000000,18,1.000000,18.00,"['', '', '', '', '', '', '', '', '', '', '', '...",{'': 18},0.196508,0.491190,0.152726,0.463984,"['AT&amp;T', '(', 'NYSE', ':', 'T', ')', '-', ...","['[', 'removed', ']', '[', 'removed', ']', '['...",0.349234,{'2020': 18},"{'12': 4, '11': 11, '10': 3}","{'2020-12-06': 4, '2020-11-28': 5, '2020-11-21..."
3,3,1-1-1-1-1-1-1-1-1-2,14,My attempt at EBITDA forecast valuation for Te...,So I've tried to estimate the 10-year return f...,"['2021-04-14', '2021-04-14', '2021-01-31', '20...",14 14 31 18 13 23 15 16 9 3 22 5 23 10,"['4', '4', '1', '11', '8', '7', '7', '6', '4',...","['2021', '2021', '2021', '2020', '2020', '2020...",['BCE'],{'BCE': 1},,16.214286,227,2.071429,29,0.553571,7.75,"['', '', '', '', '', '', '', '', '', '', '', '...",{'': 14},0.102698,0.429048,0.076278,0.386660,"['My', 'attempt', 'at', 'EBITDA', 'forecast', ...","['So', 'I', ""'ve"", 'tried', 'to', 'estimate', ...",0.178977,"{'2021': 3, '2020': 7, '2019': 4}","{'4': 3, '1': 2, '11': 3, '8': 1, '7': 2, '6':...","{'2021-04-14': 2, '2021-01-31': 1, '2020-11-18..."
4,4,10meh,13,How Hedge Against The US Dollar ? (For investo...,If You own an investment in a country other th...,"['2020-12-04', '2020-10-14', '2020-10-14', '20...",4 14 14 24 27 6 19 17 15 14 12 10 30,"['12', '10', '10', '8', '6', '6', '5', '5', '5...","['2020', '2020', '2020', '2020', '2020', '2020...",NaN,{},,34.076923,443,19.692308,256,0.965385,12.55,"['', '', '', '', '', '', '', '', '', '', '', '...",{'': 13},0.130777,0.591317,0.193169,0.438436,"['How', 'Hedge', 'Against', 'The', 'US', 'Doll...","['If', 'You', 'own', 'an', 'investment', 'in',...",0.323946,{'2020': 13},"{'12': 1, '10': 2, '8': 1, '6': 2, '5': 6, '4'...","{'2020-12-04': 1, '2020-10-14': 2, '2020-08-24..."


In [9]:
### intersect top users of all three subreddits to find common users

wsb_users = wsb['author'].tolist()
stocks_users = stocks['author'].tolist()
investing_users = investing['author'].tolist()
common_users = set(wsb_users) & set(stocks_users) & set(investing_users)
len(common_users)

33

In [21]:
def common_elements(list1,list2,list3=[]):
    users_wsb_stocks = set(list1) & set(list2) 
    print('Elements in List 1:',len(list1))
    print('Elements in List 2:',len(list2))
    print('common_elements [List 1,2]:',len(users_wsb_stocks))
    print('common_elements:',users_wsb_stocks)
    if list3:
      print('Elements in List 3:',len(list3))
      users_wsb_investing = set(list1) & set(list3)
      print('common_elements [List 1,3]:',len(users_wsb_investing))
      print('common_elements:',users_wsb_investing)
      users_stocks_investing = set(list2) & set(list3)
      print('common_elements [List 2,3]:',len(users_stocks_investing))
      print('common_elements:',users_stocks_investing)  
      users = set(list1) & set(list2) & set(list3)
      print('common_elements [List 1,2,3]:',len(users))
      print('common_elements:',users)

In [25]:
common_elements(wsb_users,stocks_users,investing_users)

Elements in List 1: 7253
Elements in List 2: 985
common_elements [List 1,2]: 143
Elements in List 3: 640
common_elements [List 1,3]: 92
common_elements [List 2,3]: 117
common_elements [List 1,2,3]: 33


In [15]:
## DD users

wsb_dd = pd.read_csv('/content/drive/MyDrive/WSB/WSB_users_DD.csv',sep=';')
stocks_dd = pd.read_csv('/content/drive/MyDrive/stocks/stocks_users_DD.csv',sep=';')
investing_dd = pd.read_csv('/content/drive/MyDrive/investing/investing_users_DD.csv',sep=';')

In [26]:
common_elements(wsb_dd['author'].tolist(),stocks_dd['author'].tolist(),investing_dd['author'].tolist())

Elements in List 1: 3486
Elements in List 2: 985
common_elements [List 1,2]: 94
Elements in List 3: 45
common_elements [List 1,3]: 3
common_elements [List 2,3]: 7
common_elements [List 1,2,3]: 1


In [18]:
## Pre and Post users

wsb_pre = pd.read_csv('/content/drive/MyDrive/WSB/WSB_users_pre-hype.csv',sep=';')
wsb_post = pd.read_csv('/content/drive/MyDrive/WSB/WSB_users_post-hype.csv',sep=';')
stocks_pre = pd.read_csv('/content/drive/MyDrive/stocks/stocks_users_pre-hype.csv',sep=';')
stocks_post = pd.read_csv('/content/drive/MyDrive/stocks/stocks_users_post-hype.csv',sep=';')
investing_pre = pd.read_csv('/content/drive/MyDrive/investing/investing_users_pre-hype.csv',sep=';')
investing_post = pd.read_csv('/content/drive/MyDrive/investing/investing_users_post-hype.csv',sep=';')

In [27]:
## WSB common Pre and Post users
common_elements(wsb_pre['author'].tolist(),wsb_post['author'].tolist())

Elements in List 1: 1034
Elements in List 2: 5919
common_elements [List 1,2]: 499


In [28]:
## stocks common Pre and post users
common_elements(stocks_pre['author'].tolist(),stocks_post['author'].tolist())

Elements in List 1: 283
Elements in List 2: 793
common_elements [List 1,2]: 204


In [29]:
## investing common Pre and Post users
common_elements(investing_pre['author'].tolist(),investing_post['author'].tolist())

Elements in List 1: 271
Elements in List 2: 335
common_elements [List 1,2]: 120


In [57]:
## find common Tickers between pre vs post, subreddits
import re
def count_freq(_column): # column type : list
    trigrams_title = []
    #print(type(_column))
    #print(_column)
    for lst in _column.tolist():  
          #print(type(lst))
          #print(lst)
          if not pd.isna(lst):
            for trigram in lst.split(','):    
              trigram=re.sub('[[]', '', trigram)
              trigram=re.sub('[]]', '', trigram)
              if trigram != '':      
                  trigrams_title.append(trigram.lower())
    count_freq_trigrams = {}
    for item in trigrams_title:
        item = item.strip()
        if item in count_freq_trigrams:
            count_freq_trigrams[item] +=1
        else:
            count_freq_trigrams[item] = 1
    return sorted(count_freq_trigrams.items(), key=lambda item: item[1], reverse = True)


In [81]:
def common_tickers(_column1,_column2,_column3=pd.Series([])): # column type : series
    tickers1 = []
    tickers2 = []
    tickers3 = []
    ticker_lst = [tickers1,tickers2,tickers3]
    columns_lst = [_column1,_column2,_column3]
    for ticker,col in zip(ticker_lst,columns_lst):
        for lst in col.tolist():  
            if not pd.isna(lst):
                for trigram in lst.split(','):    
                  trigram=re.sub('[[]', '', trigram)
                  trigram=re.sub('[]]', '', trigram)
                  if trigram != '':      
                      ticker.append(trigram.lower())
    common_elements(tickers1,tickers2,tickers3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [78]:
common_tickers(wsb_dd['tickers_'],stocks_dd['tickers_'],investing_dd['tickers_'])

Elements in List 1: 6379
Elements in List 2: 1703
common_elements [List 1,2]: 669
common_elements: {" 'iclk'", " 'ljpc'", " 'mac'", " 'net'", " 'roku'", " 'sono'", " 'vtrs'", " 'rtx'", " 'nke'", "'ride'", " 'sgh'", " 'hbi'", " 'asts'", " 'air'", " 'smts'", " 'hrb'", " 'aep'", " 'pcg'", " 'auph'", " 'ctek'", " 'ois'", "'gild'", "'twlo'", " 'ccxi'", " 'sldb'", " 'mmm'", " 'orcl'", " 'snx'", " 'sify'", " 'tlry'", " 'arw'", "'mu'", " 'arwr'", " 'auy'", " 'scs'", " 'gpl'", " 'sjr'", " 'blnk'", " 'mga'", " 'isrg'", " 'gnk'", " 'pump'", " 'ma'", "'pep'", " 'lpx'", "'viac'", " 'ayx'", " 'sq'", " 'ng'", " 'uber'", " 'virt'", " 'nova'", " 'veev'", " 'albo'", "'amd'", " 'tcom'", "'fuv'", " 'tgtx'", " 'din'", " 'xpo'", " 'amat'", " 'grub'", " 'wcn'", " 'dgly'", " 'dal'", " 'gm'", " 'pypl'", " 'sdc'", " 'teck'", "'ba'", "'cmg'", " 'crm'", " 'panw'", " 'wfc'", "'lvs'", " 'b'", " 'cx'", " 'lgih'", "'low'", " 'rng'", " 'chra'", " 'plnt'", " 'hun'", " 'li'", " 'mara'", " 'cbsh'", " 'sail'", " 'bep'", "

In [79]:
common_tickers(wsb['tickers_'],stocks['tickers_'],investing['tickers_'])

Elements in List 1: 9406
Elements in List 2: 1703
common_elements [List 1,2]: 709
common_elements: {" 'iclk'", " 'ljpc'", " 'mac'", " 'net'", " 'smar'", " 'roku'", "'lrn'", " 'sono'", " 'vtrs'", " 'rtx'", " 'nke'", "'ride'", " 'sgh'", " 'hbi'", " 'asts'", " 'air'", " 'smts'", " 'hrb'", " 'aep'", " 'pcg'", " 'auph'", " 'ctek'", " 'ois'", "'gild'", "'twlo'", " 'ccxi'", " 'sldb'", " 'mmm'", " 'orcl'", " 'snx'", " 'sify'", " 'tlry'", " 'arw'", "'mu'", " 'arwr'", " 'auy'", "'dal'", " 'scs'", " 'gpl'", " 'sjr'", " 'blnk'", " 'mga'", " 'isrg'", " 'gnk'", " 'pump'", " 'ma'", "'pep'", " 'lpx'", "'viac'", " 'ayx'", " 'sq'", " 'ng'", " 'uber'", " 'virt'", " 'nova'", " 'veev'", " 'albo'", "'amd'", " 'tcom'", "'fuv'", " 'tgtx'", " 'din'", " 'xpo'", " 'amat'", " 'grub'", " 'wcn'", " 'dgly'", " 'dal'", " 'gm'", " 'pypl'", " 'sdc'", " 'teck'", "'ba'", "'cmg'", " 'crm'", " 'panw'", " 'wfc'", "'lvs'", " 'b'", " 'cx'", " 'lgih'", "'low'", " 'rng'", " 'chra'", " 'plnt'", " 'hun'", " 'li'", " 'mara'", " 'c

In [82]:
common_tickers(wsb_pre['tickers_'],wsb_post['tickers_'])

Elements in List 1: 1132
Elements in List 2: 6520
common_elements [List 1,2]: 358
common_elements: {"'upld'", "'seas'", " 'roku'", " 'sono'", " 'nke'", "'ride'", " 'aeo'", " 'pcg'", "'crox'", " 'm'", "'jd'", "'gild'", "'twlo'", " 'sldb'", " 'tlry'", "'mu'", " 'hpe'", "'play'", "'pep'", " 'ayx'", " 'sq'", " 'uber'", "'pfe'", " 'ally'", " 'veev'", "'amd'", "'siga'", "'bam'", "'twtr'", " 'grub'", "'aal'", "'cris'", " 'gm'", " 'pypl'", " 'blct'", "'ba'", " 'wfc'", " 'crm'", "'cmg'", "'lvs'", " 'plnt'", "'ostk'", " 'li'", " 'psth'", " 'rcl'", "'hmc'", " 'khc'", "'pcg'", "'holx'", "'hal'", " 'enph'", " 'cag'", " 'ccl'", "'pton'", " 'cvs'", "'amat'", " 'avgr'", " 'xpev'", " 'a'", "'mcd'", " 'gh'", "'pins'", " 'ttd'", " 'tho'", "'goog'", " 'clx'", " 'urbn'", " 'ilmn'", "'kr'", "'cgc'", " 'gis'", " 'dkng'", "'cei'", " 'splk'", " 'yy'", " 'hd'", " 'ci'", "'bbw'", " 'x'", " 'etsy'", " 'wynn'", " 'yum'", " 'cof'", " 'aapl'", "'expr'", " 'w'", " 'cost'", " 'dbx'", "'trip'", "'snap'", " 'v'", "'atvi

In [83]:
common_tickers(stocks_pre['tickers_'],stocks_post['tickers_'])

Elements in List 1: 179
Elements in List 2: 1236
common_elements [List 1,2]: 105
common_elements: {" 'etsy'", " 'wday'", " 'f'", " 'be'", "'zom'", " 'skx'", " 'crwd'", " 'shop'", " 'aapl'", "'acb'", " 'cost'", " 'sldb'", " 'se'", " 'chek'", " 'tdoc'", " 'wmg'", "'dkng'", " 'cocp'", " 'visl'", " 't'", " 'jpm'", "'dal'", "'hrtx'", " 'csco'", " 'bigc'", "'viac'", " 'gold'", " 'jd'", " 'ayx'", " 'sq'", " 'bac'", "'x'", " 'well'", " 'whr'", "'d'", " 'ino'", " 'bud'", "'four'", " 'bio'", "'ship'", " 'run'", " 'msft'", " 'sava'", " 'dgly'", "'nvda'", " 'swbi'", " 'pypl'", " 'sdc'", " 'ttnp'", " 'crm'", "'syy'", " 'bkyi'", " 'o'", " 'sint'", "'gold'", "'wmt'", " 'ba'", " 'plug'", " 'bynd'", "'wkhs'", "'msft'", " 'pd'", "'hal'", " 'ge'", " 'irm'", "'idex'", "'luv'", " 'spce'", " 'ccl'", "'zs'", " 'lh'", " 'apog'", " 'cvs'", " 'avgr'", "'sq'", " 'phm'", "'tsla'", " 'cvna'", "'mgm'", "'aapl'", " 'tsla'", " 'tnxp'", " 'lca'", "'spce'", " 'mcd'", " 'wmt'", "'nio'", " 'vtiq'", " 'big'", " 'lrn'", "'

In [84]:
common_tickers(investing_pre['tickers_'],investing_post['tickers_'])

Elements in List 1: 140
Elements in List 2: 243
common_elements [List 1,2]: 42
common_elements: {"'wmt'", "'cers'", " 'anat'", " 'gold'", "'ge'", " 'noc'", " 'bynd'", " 'tsla'", " 'lea'", " 'khc'", "'pfe'", " 'awk'", " 'gure'", " 'f'", " 'amzn'", " 'cat'", "'pm'", "'uber'", "'virt'", "'abbv'", " 'msft'", " 'aapl'", " 'apo'", " 'mo'", "'luv'", " 'itci'", " 'spce'", " 'ccl'", " 'dis'", "'nnox'", " 'wab'", " 'crm'", " 'adt'", " 'jpm'", " 'ko'", "'hrtx'", "'sq'", " 'gild'", " 'baba'", "'tsla'", " 'nflx'", "'all'"}


In [64]:
#WSB
print('WSB_Pre-hype_tickers:',count_freq(wsb_pre['tickers_'])[:50])
print('WSB_Post-hype_tickers:',count_freq(wsb_post['tickers_'])[:50])

WSB_Pre-hype_tickers: [("'tsla'", 104), ("'spce'", 50), ("'bynd'", 27), ("'msft'", 27), ("'dis'", 22), ("'pltr'", 22), ("'nio'", 21), ("'ba'", 21), ("'amzn'", 20), ("'aapl'", 20), ("'amd'", 19), ("'snap'", 15), ("'zm'", 14), ("'nkla'", 13), ("'fb'", 12), ("'baba'", 12), ("'pton'", 11), ("'uber'", 10), ("'gme'", 10), ("'lulu'", 10), ("'mu'", 10), ("'sbux'", 9), ("'gild'", 9), ("'roku'", 8), ("'bac'", 8), ("'cgc'", 8), ("'atvi'", 8), ("'sq'", 8), ("'ino'", 8), ("'bbby'", 8), ("'fsly'", 7), ("'cost'", 7), ("'goog'", 6), ("'nvda'", 6), ("'tgt'", 6), ("'ge'", 6), ("'lyft'", 6), ("'shop'", 5), ("'pcg'", 5), ("'mo'", 5), ("'tlry'", 5), ("'mcd'", 5), ("'twtr'", 5), ("'f'", 5), ("'acb'", 5), ("'nke'", 5), ("'mrna'", 5), ("'gm'", 5), ("'crm'", 5), ("'ulta'", 5)]
WSB_Post-hype_tickers: [("'gme'", 805), ("'amc'", 342), ("'bb'", 296), ("'tsla'", 247), ("'pltr'", 211), ("'wish'", 148), ("'clov'", 146), ("'nok'", 137), ("'spce'", 132), ("'nio'", 73), ("'rkt'", 69), ("'baba'", 66), ("'aapl'", 58), ("'

In [67]:
print('WSB_DD_tickers:',count_freq(wsb_dd['tickers_'])[:50])

WSB_DD_tickers: [("'gme'", 445), ("'tsla'", 250), ("'amc'", 212), ("'bb'", 176), ("'spce'", 129), ("'pltr'", 126), ("'wish'", 103), ("'clov'", 102), ("'nok'", 87), ("'msft'", 60), ("'aapl'", 59), ("'amzn'", 55), ("'nio'", 55), ("'baba'", 55), ("'dis'", 53), ("'amd'", 53), ("'bynd'", 47), ("'rkt'", 47), ("'ba'", 40), ("'snap'", 40), ("'wkhs'", 37), ("'tlry'", 36), ("'clne'", 33), ("'sndl'", 32), ("'clf'", 31), ("'zm'", 31), ("'sdc'", 30), ("'fb'", 30), ("'f'", 28), ("'uwmc'", 28), ("'nkla'", 27), ("'pton'", 27), ("'bbby'", 26), ("'crsr'", 26), ("'nvda'", 24), ("'dkng'", 23), ("'root'", 22), ("'acb'", 21), ("'goev'", 21), ("'hood'", 20), ("'sofi'", 20), ("'x'", 19), ("'lulu'", 19), ("'sq'", 19), ("'ino'", 19), ("'atvi'", 19), ("'mvis'", 19), ("'irnt'", 19), ("'mrna'", 19), ("'roku'", 18)]


In [65]:
#stocks
print('Stocks_Pre-hype_tickers:',count_freq(stocks_pre['tickers_'])[:50])
print('Stocks_Post-hype_tickers:',count_freq(stocks_post['tickers_'])[:50])

Stocks_Pre-hype_tickers: [("'tsla'", 9), ("'msft'", 4), ("'dkng'", 3), ("'sq'", 3), ("'dis'", 3), ("'aapl'", 3), ("'nvda'", 3), ("'spce'", 3), ("'net'", 2), ("'vtiq'", 2), ("'ba'", 2), ("'fb'", 2), ("'gold'", 2), ("'acb'", 2), ("'wkhs'", 2), ("'ge'", 2), ("'nkla'", 2), ("'nio'", 2), ("'jpm'", 2), ("'wmt'", 2), ("'csco'", 2), ("'mrna'", 2), ("'zom'", 1), ("'visl'", 1), ("'hexo'", 1), ("'dal'", 1), ("'mgm'", 1), ("'fang'", 1), ("'dgly'", 1), ("'sint'", 1), ("'pypl'", 1), ("'bset'", 1), ("'bigc'", 1), ("'ai'", 1), ("'big'", 1), ("'mcd'", 1), ("'cwh'", 1), ("'cvna'", 1), ("'hbi'", 1), ("'cvgi'", 1), ("'xpo'", 1), ("'crm'", 1), ("'dg'", 1), ("'penn'", 1), ("'lstr'", 1), ("'dell'", 1), ("'tdoc'", 1), ("'phm'", 1), ("'momo'", 1), ("'skx'", 1)]
Stocks_Post-hype_tickers: [("'tsla'", 21), ("'gme'", 14), ("'msft'", 12), ("'net'", 12), ("'dkng'", 11), ("'aapl'", 11), ("'nio'", 11), ("'spce'", 9), ("'bb'", 8), ("'fb'", 8), ("'baba'", 8), ("'amc'", 7), ("'intc'", 7), ("'sq'", 7), ("'rkt'", 7), ("'am

In [66]:
#investing
print('investing_Pre-hype_tickers:',count_freq(investing_pre['tickers_'])[:50])
print('investing_Post-hype_tickers:',count_freq(investing_post['tickers_'])[:50])

investing_Pre-hype_tickers: [("'tsla'", 7), ("'amzn'", 3), ("'qcom'", 3), ("'jpm'", 3), ("'ba'", 3), ("'nflx'", 3), ("'wmt'", 2), ("'shop'", 2), ("'sq'", 2), ("'csco'", 2), ("'fb'", 2), ("'nvda'", 2), ("'t'", 2), ("'aapl'", 2), ("'dis'", 2), ("'googl'", 2), ("'bynd'", 2), ("'uber'", 2), ("'tgt'", 1), ("'htz'", 1), ("'cers'", 1), ("'apo'", 1), ("'adt'", 1), ("'awk'", 1), ("'noc'", 1), ("'ge'", 1), ("'wab'", 1), ("'dbx'", 1), ("'spce'", 1), ("'x'", 1), ("'ptn'", 1), ("'jmia'", 1), ("'wynn'", 1), ("'amat'", 1), ("'alb'", 1), ("'sqm'", 1), ("'pm'", 1), ("'mo'", 1), ("'gild'", 1), ("'hasi'", 1), ("'mkc'", 1), ("'iq'", 1), ("'wkhs'", 1), ("'uavs'", 1), ("'all'", 1), ("'gure'", 1), ("'anat'", 1), ("'lea'", 1), ("'luv'", 1), ("'abbv'", 1)]
investing_Post-hype_tickers: [("'tsla'", 9), ("'gme'", 5), ("'amzn'", 4), ("'kulr'", 3), ("'bbig'", 3), ("'mara'", 2), ("'riot'", 2), ("'nvax'", 2), ("'sq'", 2), ("'bb'", 2), ("'luv'", 2), ("'jpm'", 2), ("'aapl'", 2), ("'sndl'", 2), ("'amc'", 2), ("'etsy'", 